In [18]:
import pyarrow as pa
import pyarrow.ipc as ipc
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
from sklearn.metrics import classification_report

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [13]:
# Chemin vers le fichier .arrow
file_path = '/Users/dominicprenovost/Programmation/AutoML/AutoML-Practice/pricing-canada.arrow'

# Lecture du fichier .arrow
with pa.memory_map(file_path, 'r') as source:
    table = ipc.RecordBatchFileReader(source).read_all()

# Conversion en DataFrame Pandas
df = table.to_pandas()

# Affichage du DataFrame
df.head()

,date,sid,market_cap_rank_quarterly,market_cap_rep,market_cap_usd,E_D_RAW_dividend_per_share_ex_date,E_D_RAW_CSHOC_shares_outstanding,E_D_RAW_CSHTRD_trading_volume_share,E_D_RAW_price_close_usd,E_D_RAW_price_open_usd,E_D_RAW_price_close_rep,E_D_RAW_price_open_rep,E_D_RAW_price_close_trd,E_D_RAW_price_close_unadj_trd,E_D_RAW_price_close_unadj_usd,E_D_ADJ_price_close_usd,E_D_RAW_split_rate,E_D_RAW_TRF,trading_value_1d_usd
0,2024-01-02,SP-00109601C,238.0,1141.312150,857.206365,0.0,10813000.0,2575.0,79.275535,78.862446,105.55000,105.000000,105.55,105.55,79.275535,79.275535,1.0,1.638521,2.041345e+05
1,2024-01-02,SP-00118601C,27.0,26753.097226,26753.097226,0.0,494378000.0,899914.0,54.114660,54.662941,54.11466,54.662941,72.05,72.05,54.114660,54.114660,1.0,1.538673,4.869854e+07
2,2024-01-02,SP-00126201C,311.0,574.939370,431.820241,0.0,38509000.0,7302.0,11.213489,11.266064,14.93000,15.000000,14.93,14.93,11.213489,112.134888,1.0,2.773217,8.188090e+04
3,2024-01-02,SP-00126307C,212.0,1320.938250,992.118305,0.0,103603000.0,81704.0,9.576154,9.839029,12.75000,13.100000,12.75,12.75,9.576154,9.576154,1.0,1.051654,7.824101e+05
4,2024-01-02,SP-00182801C,115.0,4396.648620,3302.194918,0.0,100849000.0,192105.0,29.013870,28.916230,38.63000,38.500000,38.63,38.63,29.013870,116.055478,1.0,2.692941,5.573709e+06


## Binary Classification Exemple


In [26]:
# Chemin vers le fichier .arrow
file_path = '/Users/dominicprenovost/Programmation/AutoML/AutoML-Practice/pricing-canada.arrow'

# Lecture du fichier .arrow
with pa.memory_map(file_path, 'r') as source:
    table = ipc.RecordBatchFileReader(source).read_all()

# Conversion en DataFrame Pandas
df = table.to_pandas()

#df = df.set_index('date', inplace=True)

X_train, X_test, y_train, y_test = train_test_split(
    df[df.columns[:-1]], df["trading_value_1d_usd"], test_size=0.25
)

automl = AutoML()
automl.fit(X_train, y_train)

predictions = automl.predict(X_test)

#print(classification_report(y_test, predictions))

/Users/dominicprenovost/anaconda3/envs/HECFinance/lib/python3.11/site-packages/supervised/preprocessing/exclude_missing_target.py:25: UserWarning: There are samples with missing target values in the data which will be excluded for further analysis


Linear algorithm was disabled.
AutoML directory: AutoML_5
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Baseline', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_Baseline rmse 60594611.475944 trained in 0.16 seconds
2_DecisionTree rmse 28422235.883344 trained in 4.8 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost rmse 12907268.46649 trained in 1.04 seconds
4_Default_NeuralNetwork rmse 33461371.182789 trained in 1.05 seconds
5_Default_RandomForest rmse 18086113.013295 trained in 5.85 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 11666373.238693 trained in 0.06 seconds
AutoML fit time: 15.44 seconds
AutoML best model: Ensemble


In [27]:
import pandas as pd
import pyarrow as pa
import pyarrow.ipc as ipc
from supervised.automl import AutoML
from sklearn.metrics import mean_squared_error, r2_score

# Chemin vers le fichier .arrow
file_path = '/Users/dominicprenovost/Programmation/AutoML/AutoML-Practice/pricing-canada.arrow'

# Lecture du fichier .arrow
with pa.memory_map(file_path, 'r') as source:
    table = ipc.RecordBatchFileReader(source).read_all()

# Conversion en DataFrame Pandas
df = table.to_pandas()

# Convertir 'date' en datetime et ajouter une colonne 'month'
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month

# Suppression des colonnes inutiles
df.drop(columns=['date'], inplace=True)

# Initialiser une liste pour stocker les résultats
results = []

# Obtenir les mois uniques
unique_months = sorted(df['month'].unique())

# Fenêtre glissante
for i in range(len(unique_months) - 1):
    train_months = unique_months[:i+1]  # Mois jusqu'à i inclus
    test_month = unique_months[i+1]    # Mois suivant pour le test

    # Filtrer les données pour le train et le test
    train_data = df[df['month'].isin(train_months)]
    test_data = df[df['month'] == test_month]

    # Séparer les features (X) et la cible (y)
    X_train = train_data.drop(columns=['trading_value_1d_usd', 'month'])
    y_train = train_data['trading_value_1d_usd']
    X_test = test_data.drop(columns=['trading_value_1d_usd', 'month'])
    y_test = test_data['trading_value_1d_usd']

    # Initialiser AutoML
    automl = AutoML(
        mode="Perform",
        total_time_limit=600,  # Limite pour chaque itération
        eval_metric="rmse"
    )

    # Entraîner le modèle
    automl.fit(X_train, y_train)

    # Faire des prédictions
    predictions = automl.predict(X_test)

    # Évaluer les performances
    rmse = mean_squared_error(y_test, predictions, squared=False)
    r2 = r2_score(y_test, predictions)

    # Stocker les résultats
    results.append({
        'train_months': train_months,
        'test_month': test_month,
        'rmse': rmse,
        'r2': r2
    })

    print(f"Train Months: {train_months}, Test Month: {test_month}, RMSE: {rmse}, R2: {r2}")

# Convertir les résultats en DataFrame pour les analyser
results_df = pd.DataFrame(results)
print(results_df)

AutoML directory: AutoML_6
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Linear', 'Random Forest', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'ensemble']
* Step simple_algorithms will try to check up to 1 model


/Users/dominicprenovost/anaconda3/envs/HECFinance/lib/python3.11/site-packages/supervised/preprocessing/exclude_missing_target.py:25: UserWarning: There are samples with missing target values in the data which will be excluded for further analysis


1_Linear rmse 37495195.271693 trained in 3.99 seconds (1-sample predict time 0.0294 seconds)
* Step default_algorithms will try to check up to 5 models
2_Default_LightGBM rmse 15671390.088235 trained in 12.43 seconds (1-sample predict time 0.0134 seconds)
3_Default_Xgboost rmse 18193345.891846 trained in 3.83 seconds (1-sample predict time 0.0147 seconds)
4_Default_CatBoost rmse 12688537.147082 trained in 4.18 seconds (1-sample predict time 0.0137 seconds)
5_Default_NeuralNetwork rmse 8278616.690679 trained in 1.55 seconds (1-sample predict time 0.0287 seconds)
6_Default_RandomForest rmse 27583468.237183 trained in 4.96 seconds (1-sample predict time 0.0863 seconds)
* Step not_so_random will try to check up to 20 models
11_LightGBM rmse 19356009.957686 trained in 22.87 seconds (1-sample predict time 0.0135 seconds)
7_Xgboost rmse 17057292.856503 trained in 2.72 seconds (1-sample predict time 0.0144 seconds)
15_CatBoost rmse 12841235.25239 trained in 17.61 seconds (1-sample predict time

/Users/dominicprenovost/anaconda3/envs/HECFinance/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.


ValueError: Input contains NaN.